# The Variance Estimator
Let $P_\mathbf{x} := \mathbb{P}(\mathbf{z} = \mathbf{x})$.
Let $\mathbf{z}^*$ be the realized treatment vector.
Let $P^* := \mathbb{P}(\mathbf{z} = \mathbf{z^*})$ and $P_S^* := \mathbb{P}(\mathbf{z_S} = \mathbf{z_S^*})$ for $S\subseteq[n]$.
Then, the variance estimator is given by $\widehat{\text{Var}}(\widehat{TTE}) = (*) + (**)$ where 
$$(*) = \frac{1}{n^2} \sum_{i=1}^n \sum_{j \in M_i} \frac{1}{P_{N_i \cup N_j}^*} \cdot Y_i(\mathbf{z}^*)w_i(\mathbf{z}^*) \cdot Y_j(\mathbf{z}^*)w_j(\mathbf{z}^*)\cdot \text{Cov}_{ij}$$
and
$$(**) = \frac{1}{n^2} \sum_{i=1}^n P_{N_i}^* \cdot Y_i^2\mathbf{z^*} w_i^2\mathbf{z^*} \cdot \sum_{j \in M_i} 2^{|N_j|} - 2^{|N_j\setminus N_i|}$$
with 
$$\text{Cov}_{ij} := \prod_{k \in N_i \cup N_j} p^{\mathbf{z}_k^*}(1-p)^{1-\mathbf{z}_k^*}\left(1 - \prod_{\ell \in N_i \cap N_j} p^{\mathbf{z}_\ell^*}(1-p)^{1-\mathbf{z}_\ell^*}\right).$$

## Computing Term 1
We compute the first term, $(*)$, as follows:
- First, create an $n \times 1$ array $\mathbf{YW}$ with entry $i$ equal to $Y_i(\mathbf{z}^*)w_i(\mathbf{z}^*)$.
- Next, we create an $n \times 1$ array $\mathbf{V}$ with entry $i$ equal to $$\sum_{j \in M_i} \frac{1}{P_{N_i \cup N_j}^*} \cdot Y_j(\mathbf{z}^*)w_j(\mathbf{z}^*)\cdot \text{Cov}_{ij}.$$
    -- For each $i \in [n]$, we create the following $m_i := |M_i| \times 1$ arrays: 
    $$\mathbf{P}_i := \begin{bmatrix} 1/P_{N_i \cup N_{j_1}} \\ \vdots \\ 1/P_{N_i \cup N_{j_{m_i}}} \end{bmatrix},\ 
    \mathbf{YW}_i = \begin{bmatrix} Y_{j_1}(\mathbf{z}^*)w_{j_1}(\mathbf{z}^*) \\ \vdots \\ Y_{j_{m_i}}(\mathbf{z}^*)w_{j_{m_i}}(\mathbf{z}^*) \end{bmatrix}, \
    \textbf{COV}_i := \begin{bmatrix} \text{Cov}_{i{j_1}} \\ \vdots \\ \text{Cov}_{i{j_{m_i}}} \end{bmatrix}.$$

    -- Then, entry-wise multiply these arrays to get one array, sum the entries of the resulting array, and that sum is the $i$-th entry of $\mathbf{V}$.
- Then, return the dot product of $\mathbf{YW}$ and $\mathbf{V}$ divided by $n^2$.

## Computing Term 2
We compute the second term, $(**)$, as follows:
- First, compute an $n \times 1$ array $\mathbf{PY2W2}$ where entry $i$ equals $P_{N_i}^* Y_i^2(\mathbf{z}^*) w_i^2(\mathbf{z}^*).$
- Then, compute an $n \times 1$ array $\mathbf{CNTS}$ where entry $i$ equals $$\sum_{j\in M_i} 2^{|N_j|} - 2^{|N_j \setminus N_i|}.$$
- Finally, take the dot product of $\mathbf{PY2W2}$ and $\mathbf{CNTS}$ and divide by $n^2$

## The Code

In [1]:
import numpy as np

In [2]:
def two_hop_neighbors(i,A):
    '''Returns a list or array containing the units in i's two-hop network neighborhood, M_i := {j : N_i intersect N_j is nonempty}'''
    pass

In [5]:
def var_est(n, p, y, A, z):
    '''
    n : int
        size of the population
    p : float
        treatment probability
    y : numpy array
        observations
    A : numpy array 
        adjacency matrix where (i,j)th entry = 1 iff j's treatment affects i's outcome
    z : numpy array
        realized treatment assignment vector
    '''
    zz = z/p - (1-z)/(1-p)
    w = A.dot(zz)
    YW = y * w
    YW_sq = np.square(YW)

    V = np.zeros(n)
    PY2W2 = np.zeros(n)
    CNTS = np.zeros(n)

    prob_p = np.power(np.ones(n)*p, z) 
    prob_1_minus_p = np.power(1 - np.ones(n)*p, 1 - z)
    
    for i in np.arange(n):
        Ni = np.nonzero(A[i,:])
        Mi = two_hop_neighbors(i,A)

        Pi = np.zeros(len(Mi))
        COVi = np.zeros(len(Mi))
        sum = 0
        for j in np.arange(len(Mi)):
            Nj = np.nonzero(A[Mi[j], :])
            Ni_or_Nj = np.union1d(Ni,Nj)

            # Compute Pi
            mult_p = prob_p[Ni_or_Nj]
            mult_1_minus_p = prob_1_minus_p[Ni_or_Nj]
            Pi[j] = np.prod(mult_p) * np.prod(mult_1_minus_p)

            # Compute COVi
            Ni_and_Nj = np.intersect1d(Ni,Nj)
            mult_p = prob_p[Ni_and_Nj]
            mult_1_minus_p = prob_1_minus_p[Ni_and_Nj]
            temp = np.prod(mult_p) * np.prod(mult_1_minus_p)
            COVi[j] = Pi[j] * (1 - temp)

            # Compute CNTS[i]
            Nj_minus_Ni = np.setdiff1d(Nj,Ni)
            sum = sum + (2**len(Nj) - 2**len(Nj_minus_Ni))


        # Compute V
        V[i] = np.sum(1/Pi * YW[Mi] * COVi)

        # Compute PY2W2
        mult_p = prob_p[Ni]
        mult_1_minus_p = prob_1_minus_p[Ni]
        PY2W2[i] = np.prod(mult_p) * np.prod(mult_1_minus_p) * YW_sq[i]

        # Compute CNTS
        CNTS[i] = sum
    
    term1 = (1/n**2) * np.dot(YW, V)
    term2 = (1/n**2) * np.dot(PY2W2, CNTS)
    return term1+term2



# Experiments

In [6]:
import scipy.sparse

In [ ]:
def var_bound(n, p, A, C, beta=1):
    '''
    Returns the conservative upper bound on the variance of the SNIPE(beta) estimator

    n (int): size of the population
    p (float): treatment probability
    A (scipy sparse array): adjacency matrix
    C (scipy sparse array): weighted adjacency matrix
    beta (int): degree of the potential outcomes model
    '''
    in_deg = scipy.sparse.diags(np.array(A.sum(axis=1)).flatten(),0)  # array of the in-degree of each node
    out_deg = scipy.sparse.diags(np.array(A.sum(axis=0)).flatten(),0)  # array of the out-degree of each node
    d_in = np.max(in_deg)
    d_out = np.max(out_deg)
    temp = np.max(4 * (beta**2), 1 / (p*(1-p)))
    if beta == 1:
        Ymax = np.max(scipy.sparse.diags(np.array(C.sum(axis=1)).flatten(),0))
    else:
        print("Oops... I haven't implemented this for beta>1 yet, so proceed with caution!")
        Ymax = 0
    bound = (1/n) * d_in * d_out * (Ymax**2) * (np.exp(1) * d_in * temp)**beta * (1/beta)**beta
    return bound